In [1]:
cd ..

/Users/dorislee/Desktop/Research/covid/covid19-vis


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("data/quarantine-activity-US-Apr16-long.csv")

In [4]:
import numpy as np

In [5]:
from chartlib.covid_chart import CovidChart
from chartlib.utils import create_lockdown_type, split_into_list, str2emo

In [8]:
# START modified from _ingest_usa_quarantine_df
quarantine_csv = "data/quarantine-activity-US-Apr16-long.csv"
quarantine_df = pd.read_csv(quarantine_csv)
groupcol = 'Province_State'
quarantine_df = quarantine_df.rename(columns={'State': 'Province_State', 'Effective Date': 'lockdown_date','Coverage.type':'Coverage'})
quarantine_df = quarantine_df.sort_values('Coverage', ascending=True)
quarantine_df['lockdown_type'] = quarantine_df.apply(lambda x: create_lockdown_type(x, 0), axis=1)
quarantine_df['emoji_string'] = quarantine_df.apply(lambda x: create_lockdown_type(x, 1), axis=1)
quarantine_df['lockdown_type'].replace('', np.nan, inplace=True)
quarantine_df = quarantine_df.dropna(subset=['lockdown_type'])
quarantine_df = quarantine_df.groupby(['lockdown_date', 'Province_State']).agg({
    'lockdown_type': lambda col: '; '.join(col),
    'emoji_string': lambda col: ''.join(col)
}).reset_index()
quarantine_df.loc[quarantine_df.lockdown_type=="Regional Border Closure/Visitor Quarantine","emoji_string"]="t"#bugfix
# Breaking up emoji into separate rows for vertical stacking
quarantine_df.emoji_string = quarantine_df.emoji_string.apply(split_into_list)
quarantine_df = quarantine_df.explode(column='emoji_string')
quarantine_df['Coverage'] = quarantine_df.emoji_string.apply(
    lambda x: 'Statewide' if str(x).isupper() else 'Regional'
)
quarantine_df = quarantine_df.sort_values('Coverage', ascending=False)
quarantine_df.emoji_string = quarantine_df.emoji_string.str.lower()
quarantine_df['emoji'] = quarantine_df['emoji_string'].map(str2emo)
quarantine_df['event_index'] = quarantine_df.groupby(['Province_State', 'lockdown_date']).cumcount()

quarantine_cols = [
    groupcol, 'lockdown_date', 'lockdown_type', 'emoji', 'emoji_string', 'event_index', 'Coverage'
]
# quarantine_cols = ['Province_State', 'lockdown_date', 'lockdown_type', 'emoji']
quarantine_df = quarantine_df[quarantine_cols]
# END modified from _ingest_usa_quarantine_df

In [9]:
quarantine_df

,Province_State,lockdown_date,lockdown_type,emoji,emoji_string,event_index,Coverage
0,Oregon,2/28/20,Gatherings (>250) Banned,👨‍👩‍👧‍👦,g,0,Statewide
123,Indiana,3/19/20,Closure of Schools,🎓,s,0,Statewide
113,Alabama,3/18/20,"Gatherings (>50) Banned, Closure of Schools",👨‍👩‍👧‍👦,g,0,Statewide
113,Alabama,3/18/20,"Gatherings (>50) Banned, Closure of Schools",🎓,s,1,Statewide
114,Alaska,3/18/20,"Closure of Restaurants, Non-essential Businesses",🍔,r,0,Statewide
...,...,...,...,...,...,...,...
190,Kansas,3/25/20,Regional Stay-at-home Order; Regional Stay-at-...,🏠,l,2,Regional
190,Kansas,3/25/20,Regional Stay-at-home Order; Regional Stay-at-...,🏠,l,3,Regional
191,Maine,3/25/20,Regional Stay-at-home Order,🏠,l,0,Regional
192,Missouri,3/25/20,Regional Stay-at-home Order; Regional Stay-at-...,🏠,l,3,Regional


In [12]:
# quarantine_df[quarantine_df.lockdown_type=="Regional Border Closure/Visitor Quarantine"]["Travel Restrictions"].values

In [13]:
# quarantine_df[quarantine_df.lockdown_type=="Regional Border Closure/Visitor Quarantine"].apply(lambda x: create_lockdown_type(x,1),axis=1)

In [8]:
odf = pd.read_csv("data/quarantine-activity-US-Apr16.csv")

In [14]:
odf.head()

,Timestamp,State,Effective Date,Coverage,State of Emergency Declaration,Travel Restrictions,Shelter-in-place Order,Gathering Limitations,Banning Gatherings of a Certain Size,K-12 School Closure,Bar and Dine-in Restaurant Closure,Non-essential Businesses Closure,Details (if any),Reference links
0,4/14/2020 18:52:13,Oregon,02-28-2020,State-wide,NaN,NaN,NaN,Banned gatherings of a certain size,250.0,NaN,NaN,NaN,"""On February 28...She later ordered the cancel...",https://en.wikipedia.org/wiki/2020_coronavirus...
1,4/13/2020 11:42:13,Washington,02-29-2020,State-wide,State of Emergency declared,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""On February 29, Governor Jay Inslee declared ...",https://en.wikipedia.org/wiki/U.S._state_and_l...
2,4/11/2020 10:57:33,Florida,03-01-2020,State-wide,State of Emergency declared,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""On March 1, Governor DeSantis declared a publ...",https://en.wikipedia.org/wiki/2020_coronavirus...
3,4/10/2020 14:39:02,Alabama,03-02-2020,State-wide,NaN,NaN,NaN,Suggested social distancing,NaN,NaN,NaN,NaN,"""On March 2, the Alabama Department of Public ...",https://en.wikipedia.org/wiki/2020_coronavirus...
4,4/10/2020 16:23:57,California,03-04-2020,State-wide,State of Emergency declared,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""On March 4, when 53 cases had been confirmed ...",https://en.wikipedia.org/wiki/U.S._state_and_l...


In [15]:
df

,State,Coverage.type,Coverage.location,Effective Date,State of Emergency Declaration,Travel Restrictions,Shelter-in-place Order,Gathering Limitations,Banning Gatherings of a Certain Size,K-12 School Closure,Bar and Dine-in Restaurant Closure,Non-essential Businesses Closure,Details (if any),Reference links,Timestamp,population_size,population_reference
0,Alabama,State-wide,Alabama,3/13/20,State of Emergency declared,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://en.wikipedia.org/wiki/U.S._state_and_l...,2020-04-10,4903185,https://www.census.gov/data/datasets/time-seri...
1,Alabama,State-wide,Alabama,3/27/20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Some (cherry-picked) businesses closed,"On March 27, all non-essential businesses"" in ...",https://en.wikipedia.org/wiki/2020_coronavirus...,2020-04-10,4903185,https://www.census.gov/data/datasets/time-seri...
2,Alabama,State-wide,Alabama,3/2/20,NaN,NaN,NaN,Suggested social distancing,NaN,NaN,NaN,NaN,"On March 2, the Alabama Department of Public H...",https://en.wikipedia.org/wiki/2020_coronavirus...,2020-04-10,4903185,https://www.census.gov/data/datasets/time-seri...
3,Alabama,State-wide,Alabama,4/4/20,NaN,NaN,Shelter-in-place order,NaN,NaN,NaN,NaN,NaN,"On April 3, Governor Ivey issued a statewide s...",https://en.wikipedia.org/wiki/2020_coronavirus...,2020-04-10,4903185,https://www.census.gov/data/datasets/time-seri...
4,Alabama,State-wide,Alabama,3/18/20,NaN,NaN,NaN,Banned gatherings of a certain size,50.0,Schools closed,NaN,NaN,"On March 12, all schools were closed from Marc...",https://en.wikipedia.org/wiki/2020_coronavirus...,2020-04-10,4903185,https://www.census.gov/data/datasets/time-seri...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,Wyoming,State-wide,Wyoming,3/15/20,NaN,NaN,NaN,NaN,NaN,Schools closed,NaN,NaN,"On March 15, Governor Gordon and Wyoming State...",https://en.wikipedia.org/wiki/U.S._state_and_l...,2020-04-13,578759,https://www.census.gov/data/datasets/time-seri...
603,Wyoming,Regional,Teton,3/28/20,NaN,NaN,Shelter-in-place order,NaN,NaN,NaN,NaN,NaN,The Town of Jackson on Saturday ordered reside...,https://trib.com/news/local/casper/jackson-iss...,2020-04-13,23464,https://www.census.gov/data/datasets/time-seri...
604,Wyoming,Regional,Teton,3/30/20,NaN,NaN,NaN,Suggested limiting of gatherings,NaN,NaN,NaN,NaN,New health order limits gatherings to househol...,https://buckrail.com/new-health-order-limits-g...,2020-04-13,23464,https://www.census.gov/data/datasets/time-seri...
605,Wyoming,Regional,Jackson,3/28/20,NaN,NaN,Shelter-in-place order,NaN,NaN,NaN,NaN,NaN,The Town of Jackson on Saturday ordered reside...,https://trib.com/news/local/casper/jackson-iss...,2020-04-13,10429,"https://en.wikipedia.org/wiki/Jackson,_Wyoming"
